In [1]:
import numpy as np

#定义PCA算法               
def PCA(data,r):
    data=np.float32(np.mat(data))
    rows,cols=np.shape(data)
    data_mean=np.mean(data,0)#对列求平均值
    A=data-np.tile(data_mean,(rows,1))#将所有样例减去对应均值得到A
    C=A*A.T #得到协方差矩阵
    D,V=np.linalg.eig(C)#求协方差矩阵的特征值和特征向量
    V_r=V[:,0:r]#按列取前r个特征向量
    V_r=A.T*V_r#小矩阵特征向量向大矩阵特征向量过渡
    for i in range(r):
        V_r[:,i]=V_r[:,i]/np.linalg.norm(V_r[:,i])#特征向量归一化
    
    final_data=A*V_r
    return final_data,data_mean,V_r

In [66]:
import matplotlib.image as mpig
import os
from sklearn.model_selection import GridSearchCV, train_test_split

# 把图片转化成一个一维的向量
def img_to_vector_mpig(img_path):
    img = mpig.imread(img_path)
    rows, cols = img.shape
    img_vector = np.zeros((1, rows*cols))
    img_vector = np.reshape(img, (1, rows*cols))
    return img_vector

# 图片文件夹
img_path_base  = "/Users/zhouwencheng/Desktop/Grass/05 AI/05 jupyter/10 data/03 图片/01 ORL"
# 文件夹下图片名称
img_file_list = os.listdir(img_path_base)

# 40个样本 index
indices = [ i for i in range(40)] 

def read_img(test_size = 0.8):
    # 总共有40个人 每个人10张照片 
    x_train = []
    y_train = [] 
    x_test  = []
    y_test  = []
    
    k = 10*test_size
    
    for p in range(40):
        for index in range(10):
            path = img_path_base + '/' + ('s%ld_%ld.bmp'%(p+1, index+1))
            img_vector = img_to_vector_mpig(path)
            img_vector = img_vector.reshape(img_vector.size, )
            if index >= k:
                x_train.append(img_vector)
                y_train.append(p)
            else:
                x_test.append(img_vector)
                y_test.append(p)
    
    
    
    x_train = np.array(x_train) 
    x_test = np.array(x_test)
    y_train = np.array(y_train) 
    y_test = np.array(y_test)  
    
    x_train = x_train.reshape(x_train.shape[0], -1)
    x_test = x_test.reshape(x_test.shape[0], -1)
     
    return x_train, y_train, x_test, y_test
 
x_train, y_train, x_test, y_test = read_img(test_size = 0.2)

print(x_test.shape)
print('OK')  

(80, 10304)
OK


In [47]:
x_test.shape

(80, 41216)

In [37]:
x_test

array([[ 39,  44,  53, ...,  29,  26,  29],
       [ 34,  34,  33, ...,  37,  40,  33],
       [ 35,  36,  37, ..., 144, 147, 143],
       ...,
       [ 75,  77,  75, ..., 116, 114, 113],
       [119, 121, 123, ...,  36,  40,  36],
       [127, 131, 128, ...,  94,  84,  89]], dtype=uint8)

## PCA

In [1]:
# PCA
import numpy as np

def zero_mean(data_mat):
    """
    零均值化
    """
    mean_val = np.mean(data_mat, axis = 0) #按列求均值，即求各个特征的均值
    new_data = data_mat - mean_val
    return new_data, mean_val

def pca(data_mat, n):
    new_data, mean_val = zero_mean(data_mat)
    cov_mat = np.cov(new_data, rowvar = 0)  #求协方差矩阵,return ndarray；若rowvar非0，一列代表一个样本，为0，一行代表一个样本
    
    eig_vlas, eig_vects = np.linalg.eig(np.mat(cov_mat)) #求特征值和特征向量,特征向量是按列放的，即一列代表一个特征向量
    eig_val_indice = np.argsort(eig_vlas) #对特征值从小到大排序
    n_eig_val_indice=eig_val_indice[-1:-(n+1):-1]   #最大的n个特征值的下标( [::-1]是逆序的意思)
    n_eig_vects = eig_vects[:,n_eig_val_indice] #最大的n个特征值对应的特征向量
    low_data_mat=new_data*n_eig_vects               #低维特征空间的数据(点乘)
    recon_mat = (low_data_mat * n_eig_vects.T) + mean_val #重构数据
    return low_data_mat, recon_mat  

In [76]:
# 人脸识别

def face_rec1():
    for r in range(10, 41, 10):
        print("当降维到%d时"%(r))
        x_value=[]
        y_value=[]
        
        for k in range(1,10): 
            test_size = 1 - k * 0.1
            train_face,train_label,test_face,test_label=read_img(test_size) 
            #利用PCA算法进行训练
            data_train_new,data_mean,V_r=PCA(train_face,r)
            num_train = data_train_new.shape[0]#训练脸总数
            num_test = test_face.shape[0]#测试脸总数
            temp_face = test_face - np.tile(data_mean,(num_test,1))
            data_test_new = temp_face*V_r #得到测试脸在特征向量下的数据
            data_test_new = np.array(data_test_new) # mat change to array
            data_train_new = np.array(data_train_new)
    
            #测试准确度
            true_num = 0
            for i in range(num_test):
                testFace = data_test_new[i,:]
                diffMat = data_train_new - np.tile(testFace,(num_train,1))#训练数据与测试脸之间距离
                sqDiffMat = diffMat**2
                sqDistances = sqDiffMat.sum(axis=1)#按行求和 
                sortedDistIndicies = sqDistances.argsort()#对向量从小到大排序，使用的是索引值,得到一个向量
                indexMin = sortedDistIndicies[0]#距离最近的索引
                if train_label[indexMin] == test_label[i]:
                    true_num += 1
                else:
                    pass

            accuracy = float(true_num)/num_test
            x_value.append(k)
            y_value.append(round(accuracy,2))
            
            print ('当每个人选择%d张照片进行训练时，The classify accuracy is: %.2f%%'%(k,accuracy * 100))
          

            
print('OK')



OK


In [77]:
face_rec1()

当降维到10时
当每个人选择1张照片进行训练时，The classify accuracy is: 59.44%
当每个人选择2张照片进行训练时，The classify accuracy is: 71.56%
当每个人选择3张照片进行训练时，The classify accuracy is: 82.50%
当每个人选择4张照片进行训练时，The classify accuracy is: 85.83%
当每个人选择5张照片进行训练时，The classify accuracy is: 88.50%
当每个人选择6张照片进行训练时，The classify accuracy is: 93.75%
当每个人选择7张照片进行训练时，The classify accuracy is: 94.17%
当每个人选择8张照片进行训练时，The classify accuracy is: 97.50%
当每个人选择9张照片进行训练时，The classify accuracy is: 95.00%
当降维到20时
当每个人选择1张照片进行训练时，The classify accuracy is: 64.72%
当每个人选择2张照片进行训练时，The classify accuracy is: 75.94%
当每个人选择3张照片进行训练时，The classify accuracy is: 86.43%
当每个人选择4张照片进行训练时，The classify accuracy is: 90.00%
当每个人选择5张照片进行训练时，The classify accuracy is: 94.00%
当每个人选择6张照片进行训练时，The classify accuracy is: 95.62%
当每个人选择7张照片进行训练时，The classify accuracy is: 96.67%
当每个人选择8张照片进行训练时，The classify accuracy is: 97.50%
当每个人选择9张照片进行训练时，The classify accuracy is: 95.00%
当降维到30时
当每个人选择1张照片进行训练时，The classify accuracy is: 66.39%
当每个人选择2张照片进行训练时，The classify accuracy is: 75.

In [69]:
#人脸识别
def face_rec():
    #k=int(input("每个人选择几张照片进行训练："))
    #x_value=[]
    #y_value=[]
    for r in range(10,41,10):
        print("当降维到%d时"%(r))
        x_value=[]
        y_value=[]
        for k in range(1,10):
#             train_face,train_label,test_face,test_label=load_orl(k)#得到数据集
            test_size = 1 - k * 0.1
            train_face,train_label,test_face,test_label=read_img(test_size) 
            #利用PCA算法进行训练
            data_train_new,data_mean,V_r=PCA(train_face,r)
            num_train = data_train_new.shape[0]#训练脸总数
            num_test = test_face.shape[0]#测试脸总数
            temp_face = test_face - np.tile(data_mean,(num_test,1))
            data_test_new = temp_face*V_r #得到测试脸在特征向量下的数据
            data_test_new = np.array(data_test_new) # mat change to array
            data_train_new = np.array(data_train_new)
    
            #测试准确度
            true_num = 0
            for i in range(num_test):
                testFace = data_test_new[i,:]
                diffMat = data_train_new - np.tile(testFace,(num_train,1))#训练数据与测试脸之间距离
                sqDiffMat = diffMat**2
                sqDistances = sqDiffMat.sum(axis=1)#按行求和 
                sortedDistIndicies = sqDistances.argsort()#对向量从小到大排序，使用的是索引值,得到一个向量
                indexMin = sortedDistIndicies[0]#距离最近的索引
                if train_label[indexMin] == test_label[i]:
                    true_num += 1
                else:
                    pass

            accuracy = float(true_num)/num_test
            x_value.append(k)
            y_value.append(round(accuracy,2))
            
            print ('当每个人选择%d张照片进行训练时，The classify accuracy is: %.2f%%'%(k,accuracy * 100))
        
        #绘图
        if r==10:
            y1_value=y_value
            plt.plot(x_value,y_value,marker="o",markerfacecolor="red")
            for a, b in zip(x_value, y_value):  
                plt.text(a,b,(a,b),ha='center', va='bottom', fontsize=10)  
 
      
            plt.title("降到10维时识别率",fontsize=14)
            plt.xlabel("K值",fontsize=14)
            plt.ylabel("识别率",fontsize=14)
            plt.show()
            #print ('y1_value',y1_value)
        if r==20:
            y2_value=y_value
            plt.plot(x_value,y2_value,marker="o",markerfacecolor="red")
            for a, b in zip(x_value, y_value):  
                plt.text(a,b,(a,b),ha='center', va='bottom', fontsize=10)  
 
      
            plt.title("降到20维时识别率",fontsize=14)
            plt.xlabel("K值",fontsize=14)
            plt.ylabel("识别率",fontsize=14)
            plt.show() 
            #print ('y2_value',y2_value)
        if r==30:
            y3_value=y_value
            plt.plot(x_value,y3_value,marker="o",markerfacecolor="red")
            for a, b in zip(x_value, y_value):  
                plt.text(a,b,(a,b),ha='center', va='bottom', fontsize=10)  
 
      
            plt.title("降到30维时识别率",fontsize=14)
            plt.xlabel("K值",fontsize=14)
            plt.ylabel("识别率",fontsize=14)
            plt.show()
            #print ('y3_value',y3_value)
        if r==40:
            y4_value=y_value 
            plt.plot(x_value,y4_value,marker="o",markerfacecolor="red")
            for a, b in zip(x_value, y_value):  
                plt.text(a,b,(a,b),ha='center', va='bottom', fontsize=10)  
 
      
            plt.title("降到40维时识别率",fontsize=14)
            plt.xlabel("K值",fontsize=14)
            plt.ylabel("识别率",fontsize=14)
            plt.show()
            #print ('y4_value',y4_value) 
        
        
        
    #各维度下准确度比较
    L1,=plt.plot(x_value,y1_value,marker="o",markerfacecolor="red")   
    L2,=plt.plot(x_value,y2_value,marker="o",markerfacecolor="red")
    L3,=plt.plot(x_value,y3_value,marker="o",markerfacecolor="red")
    L4,=plt.plot(x_value,y4_value,marker="o",markerfacecolor="red")
    #for a, b in zip(x_value, y1_value):  
    #    plt.text(a,b,(a,b),ha='center', va='bottom', fontsize=10)  
 
    plt.legend([L1,L2,L3,L4], ["降到10维", "降到20维","降到30维","降到40维"], loc=4)
    plt.title("各维度识别率比较",fontsize=14)
    plt.xlabel("K值",fontsize=14)
    plt.ylabel("识别率",fontsize=14)
    plt.show()

In [70]:
face_rec()

当降维到10时
当每个人选择1张照片进行训练时，The classify accuracy is: 59.44%
当每个人选择2张照片进行训练时，The classify accuracy is: 71.56%
当每个人选择3张照片进行训练时，The classify accuracy is: 82.50%
当每个人选择4张照片进行训练时，The classify accuracy is: 85.83%
当每个人选择5张照片进行训练时，The classify accuracy is: 88.50%
当每个人选择6张照片进行训练时，The classify accuracy is: 93.75%
当每个人选择7张照片进行训练时，The classify accuracy is: 94.17%
当每个人选择8张照片进行训练时，The classify accuracy is: 97.50%
当每个人选择9张照片进行训练时，The classify accuracy is: 95.00%


NameError: name 'plt' is not defined